In [ ]:
import wandb
import os

project_name = "CANDID_DAC"

# download all runs from a project
api = wandb.Api()
# there is a little bug in the API that might return duplicate runs but omit others, that is why we
# have to use order='-heartbeat_at' (you might need play around with that as different settings seem to work for different people )
runs = api.runs(project_name, order='-heartbeat_at')

distinct_run_ids = set()
duplicate_run_ids = set()

print(f"Number of runs (total): {len(runs)}")

for run in runs:
    if run.id in distinct_run_ids:
        duplicate_run_ids.add(run.id)
    else:
        distinct_run_ids.add(run.id)

# due to the aforementioned bug, we also check that there are no duplicate run ids
print(f"Number of distinct run ids: {len(distinct_run_ids)}")
print(f"Number of duplicate run ids: {len(duplicate_run_ids)}")

In [ ]:
import pandas as pd
# create a directory "run_updated_metrics" if it does not exist
if not os.path.exists("run_data"):
    os.makedirs("run_data")

# define paths to store the metrics and the run configuration
metrics_path = f'run_data/{project_name}_metrics.csv'
config_path = f'run_data/{project_name}_configs.csv'

# load the project data from a csv file if it already exists
if os.path.exists(metrics_path):
    existing_metrics = pd.read_csv(metrics_path)
    existing_configs = pd.read_csv(config_path)
    print(f'Found {len(existing_configs)} existing runs')
else:
    existing_metrics = pd.DataFrame()
    existing_configs = pd.DataFrame()

In [ ]:
project_run_ids = set([])

for run in runs:
    if run.Group == "reproduce":
        continue
    project_run_ids.add(run.id)
print(f'Found {len(project_run_ids)} unique runs in the project to download')

In [ ]:
import pandas as pd

# remove all runs that from the existing metrics that are not in the project run ids
if len(existing_metrics) > 0:
    existing_metrics = existing_metrics[existing_metrics['run_id'].isin(project_run_ids)]
if len(existing_configs) > 0:
    existing_configs = existing_configs[existing_configs['run_id'].isin(project_run_ids)]

existing_metric_ids = set(existing_metrics['run_id'].unique())
existing_config_ids = set(existing_configs['run_id'].unique())

run_ids_to_reload = set([])
# also remove metrics of runs to reload
if len(existing_metrics) > 0:
    existing_metrics = existing_metrics[~existing_metrics['run_id'].isin(run_ids_to_reload)]
reload_all = False

# if reload_all, add all run ids to the list of run_ids_to_reload
if reload_all:
    run_ids_to_reload = existing_metric_ids

run_ids_to_download = project_run_ids - (existing_metric_ids - run_ids_to_reload)
already_downloaded = len(existing_metric_ids) - len(run_ids_to_reload)


to_download = len(run_ids_to_download)

print(f"Already downloaded: {already_downloaded}, to download: {to_download} (reloads: {len(run_ids_to_reload)})")

**get all runs from a wandb project and extract steps and avg episod reward into a dataframe, mark all data with the run id**


In [ ]:
downloading = 1

new_metrics = []
new_config_dict = []

# remove all metrics that should be reloaded
if len(run_ids_to_reload) > 0:
    existing_metrics = existing_metrics[~existing_metrics['run_id'].isin(run_ids_to_reload)]
    existing_configs = existing_configs[~existing_configs['run_id'].isin(run_ids_to_reload)]

not_finished = 0
for run in runs:
    if run.Group == "reproduce" or run.id not in run_ids_to_download:
        continue
    run_metrics = []
    print(f"downloading run {downloading}/{to_download} ({run.name})", end="\r")

    if not run.state == "finished":
        not_finished += 1
        downloading += 1
        continue

    # get the metrics from the run
    run_metrics.append(run.history(keys=["_step", "avg_episodic_reward"], samples=100000))
    if "avg_reward_train_set" in run.summary_metrics:
        run_metrics.append(run.history(keys=["_step", "avg_reward_train_set"], samples=100000))
    if "avg_reward_test_set" in run.summary_metrics:
        run_metrics.append(run.history(keys=["_step", "avg_reward_test_set"], samples=100000))
    # merge the dataframes on the step column, using an outer join
    data_run = pd.merge(run_metrics[0], run_metrics[1], on="_step", how="outer").merge(run_metrics[2], on="_step", how="outer")
    data_run["run_id"] = run.id

    new_metrics.append(pd.DataFrame(data_run))
    
    # get the config from the run
    config = run.config
    config["run_id"] = run.id
    config["run_name"] = run.name
    new_config_dict.append(config)
    
    downloading += 1
print(f"Downloaded {downloading-1} runs, {not_finished} runs were not finished.")

**some checks before saving the data**

In [ ]:
updated_metrics = pd.concat([existing_metrics] + new_metrics, ignore_index=True)

In [ ]:
print(f"Unique run ids: {len(updated_metrics['run_id'].unique())}")
steps_per_id = updated_metrics.groupby('run_id').agg({'_step': 'max'})
deviating_ids = steps_per_id[steps_per_id['_step'] != 100000.0]
print(f"Number of ids with deviating number of steps: {len(deviating_ids)}")

In [ ]:
# single data points might get lost during download but make sure the final performance is available
steps_per_id[steps_per_id['_step'] != 100000]

In [ ]:
updated_configs = pd.concat([existing_configs, pd.DataFrame(new_config_dict)], ignore_index=True)
print(f"Unique run ids in the config: {len(updated_configs['run_id'].unique())}")

**If everything is fine store the data.**

In [ ]:
# store the avg episodic reward data to a csv file
import time

updated_metrics.to_csv(metrics_path, index=False)
print(f"Stored {len(updated_metrics)} metric values to {metrics_path} at {time.strftime('%H:%M:%S')}")
# store the config data to a csv file
updated_configs.to_csv(config_path, index=False)
print(f"Stored {len(updated_configs)} runs to {config_path} at {time.strftime('%H:%M:%S')}")